## Import necessary modules

Every Jupyter Notebook requires the path to the KineticAssembly_AD modules (.py files in the root directory) to be mentioned. This can be done by adding the path to the 'PATH' variable of the system environment. 

Additonal modules are also imported which are required to run any analysis.

In [1]:
# make sure jupyter path is correct for loading local moudules
import sys
path_to_repo = "C:\\Users\\denys\\AMGEN\\"
#Insert your path here
# path_to_repo=""
sys.path.append(path_to_repo)


import copy
from KineticAssembly_AD import ReactionNetwork, VectorizedRxnNet, VecSim    #Import required python modules explicitly
import networkx as nx
import torch
from torch import DoubleTensor as Tensor

## Setup Reaction Network
Before we begin to run a simulation, we need to create a Reaction Network that stores all the parameters required to run a simulation and other routines. The Reaction Network can be created by reading an input file. More information on how to create an input file can be found in the User Guide. 

Here a simple trimer model is used to run a simulation.
#### Read the corresponding input file and call the ReactionNetwork class

In [2]:
base_input = './tetramer_diversification.pwr'
rn = ReactionNetwork(base_input, one_step=True)
rn.resolve_tree()

['default_assoc', 1.0]
['rxn_coupling', True]
True
['monomer_add_only', False]
[(0, {'struct': <networkx.classes.graph.Graph object at 0x000001B89B59F898>, 'copies': tensor([100.], dtype=torch.float64), 'subunits': 1}), (1, {'struct': <networkx.classes.graph.Graph object at 0x000001B89ADDF518>, 'copies': tensor([100.], dtype=torch.float64), 'subunits': 1}), (2, {'struct': <networkx.classes.graph.Graph object at 0x000001B8939D7860>, 'copies': tensor([100.], dtype=torch.float64), 'subunits': 1}), (3, {'struct': <networkx.classes.graph.Graph object at 0x000001B8939D7550>, 'copies': tensor([100.], dtype=torch.float64), 'subunits': 1})]
New node added - Node index: 4 ; Node label: AM 
New node added - Node index: 5 ; Node label: AB 
New node added - Node index: 6 ; Node label: AS 
New node added - Node index: 7 ; Node label: BM 
New node added - Node index: 8 ; Node label: MS 
New node added - Node index: 9 ; Node label: ABM 
New node added - Node index: 10 ; Node label: AMS 
New node added

## Checking reaction network

The ReactionNetwork is a networkx object which creates a graph network with each node as species that can be present in the system according to the binding rules given in the input file. Each node has a unique index number that can be used to access attributes stored for that species. Each edge represents a reaction and is associated with a unique reaction_id, on and off rates and the dG value for that reaction.


After creating a Reaction Network we can looping over all network nodes to check if all species are created
Creating a dictionary for later reference. This dictionary holds the reactants as keys and values as the reaction index

In [4]:
uid_dict = {}
sys.path.append("../")
import numpy as np
from reaction_network import gtostr

print("Species present in the Reaction Network: ")
print("%3s  %2s  %2s" %("Index","--",'Species'))

nodes_list = []
for n in rn.network.nodes():
    print("%3d  %4s  %-6s" %(n,"--",gtostr(rn.network.nodes[n]['struct'])))
    nodes_list.append(gtostr(rn.network.nodes[n]['struct']))
    for k,v in rn.network[n].items():
        uid = v['uid']
        r1 = set(gtostr(rn.network.nodes[n]['struct']))
        p = set(gtostr(rn.network.nodes[k]['struct']))
        r2 = p-r1
        reactants = (r1,r2)
        uid_dict[(n,k)] = uid

print()
print("Total Number of Reactions: ",rn._rxn_count)
print("Total Number of Species: ",len(rn.network.nodes()))
        
# Dictionary that stores source,destination of an edge and maps it to its unique id
#Key : (First Reactant, Product)
#Value : (Reaction_id)
print()
print(uid_dict)

ModuleNotFoundError: No module named 'reaction_network'

## Set Initial conditions for the association rates
The next step is to define the initial conditions for the simulation. The initial concentrations are specified from the input file. However, the initial value of the association rates can be specified either through the input file 

From the user_input file, currently the code only allows 1 value to be read (from default_assoc parameter).

To set starting rates to different values the next code block takes in a list/array of all rxn rates and updates them in the reaction network object.

In [ ]:
#Define an empty torch tensor with length equal to number of reactions
new_kon = torch.zeros([rn._rxn_count], requires_grad=True).double()

#To set individual rates to different values, we need to create an list/array with different values.
length = rn._rxn_count
min_val = 0.1
max_val = 3.0
init_val = []

for i in range(length):
    # Linearly interpolate the current maximum from min_val up to max_val
    current_max = min_val + (i / (length - 1)) * (max_val - min_val)
    # Draw a random float uniformly between min_val and current_max
    val = np.random.uniform(min_val, current_max)
    init_val.append(val)

#Else we could assign all initial values to be equal to 1; performs bad for lower indeces
#init_val = 1


new_kon = new_kon + Tensor(init_val)
#for init_val = 1
#new_kon = new_kon + Tensor([init_val])
print("Len of new_kon", len(new_kon))
update_kon_dict = {}
for edge in rn.network.edges:
    update_kon_dict[edge] = new_kon[uid_dict[edge]]

nx.set_edge_attributes(rn.network,update_kon_dict,'k_on')
for edge in rn.network.edges:
    print(rn.network.get_edge_data(edge[0],edge[1]))

## Define the VectorizedRxnNet class
This takes the corresponding rxn network class as input and creates tensor arrays for all parameters values required for running a simulation.



In [ ]:
vec_rn = VectorizedRxnNet(rn, dev='cpu')

vec_rn.reset()

## Create the VecSim class to run a simulation
Takes the VecRxnNet  object as input. The simulation runtime is set by the 'runtime' argument. (Units - sec)

In [ ]:
runtime = 1e4
sim = VecSim(vec_rn, runtime, device='cpu')


# Run a Simulation
### Input Parameters: 
conc_scale: Controls the conc step at each iteration. Since the numerical integration is not performed over fixed time steps but over fixed conc. steps. For e.g. for a value of 1uM, at each iteration step a total of app. 1uM is reacted (includes all species). Can be run using the default value. A general rule is use conc_scale = 0.01 * Max_yield

conc_thresh: This can be used to periodically decrease the conc_scale parameter. After each iteration if the conc_scale is greater than the conc_thresh, then the conc_scale is decreased by mod_factor. Can be run using the default value. 

mod_bool: This argument is necessary to fix the mass balance criteria. Sometimes if the conc_scale is large, then the simulation can lead to a higher consumption of a particular species which is very low in conc, and create more of this species out of nothing. Default value:True

verbose : Print output and progress of simulation

yield_species : The species whose yield is to be tracked and returned to the optimzer. 



In [ ]:
y = sim.simulate(conc_scale=1e-2,conc_thresh=1e-2,mod_bool=True,verbose=True,yield_species=11)


In [ ]:
print(vec_rn.max_subunits)
print(vec_rn.copies_vec)


## Plot the conc. of all species vs time

We can plot the concentration vs time data for all species. We can choose which species concentration is to be plotted using the nodes_list variable.

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline
fig, ax = plt.subplots()

#Choose which species needs to be plotted
#nodes_list = ['A','B','C', 'D', 'AB','BC','AC','ABCD']

sim.plot_observable(nodes_list, ax=ax,legend=False,seed=201,lw=5)
ax.set_title("runtime: " + str(runtime) + " seconds")
handles,labels = ax.get_legend_handles_labels()
ax.set_xscale("log")
fig.legend(handles,nodes_list,loc='upper center',fancybox=True,ncol=3,fontsize='small',markerscale=1.0)
ax.grid(which="major",axis="both")

### The following code can be used to store the Conc vs Time data for the final complex

It can also be modified to store data for all species

In [ ]:
def convert_time_interval(time,conc,time_int=0.1):
    start_time=time[0]
    time_array = []
    conc_array = []
    for i in range(len(time)):
        new_time=time[i]
        ts = new_time/start_time
        if ts>=time_int:
            time_array.append(time[i])
            conc_array.append(conc[i])
            start_time=new_time
    return(time_array,conc_array)
        
    


Before storing, the data can be smoothened by selecting only a fixed separation of time intervals to store. Its useful in removing rougher area which can arise due to a high conc_scale parameter 

In [ ]:


time_arr = np.array(sim.steps)
complx_conc = np.array(sim.observables[6][1]) # chnage first bracket based on #mer and based on topology

sel_time = (time_arr >= 1e-4)
sel_indx = np.argwhere(sel_time)[0][0]

filter_time,filter_conc = convert_time_interval(time_arr[sel_indx-1:],complx_conc[sel_indx-1:],time_int=1.15)
final_time = np.concatenate((time_arr[:sel_indx],filter_time[:]))
final_conc = np.concatenate((complx_conc[:sel_indx],filter_conc[:]))

In [ ]:
with open("Conc_Profile_Trimer","w") as fl:
    fl.write("#Timestep\tConc\n")
    for i in range(len(final_time)):
        fl.write("%10.9f\t%10.9f \n" %(final_time[i],final_conc[i]))
        

### Calculate Trapping Factor

Using the cleaned up time and concentration data, we calculate the differential - dC/dln(t) and plot it. 


In [ ]:
def calc_slope(time,conc,mode='delta'):
        #There are 3 modes to calc slopes
        #Mode 1 - "delta" : mode which is just ratio of finite differences
        #Mode 2 - "log" : Gradient calc using numpy gradient function, but time is in logspace
        #Mode 3: "regular" : Gradient calc with time in normal space

        if mode=="delta":
            slopes=[]
            for i in range(len(time)-1):
                delta_c = conc[i+1]=conc[i]
                delta_t = np.log(time[i+1]-time[i])

                s = delta_c/delta_t
                slope.append(s)

            return(slopes)
        elif mode=='log':
            l_grad = np.gradient(conc,np.log(time))
            return(l_grad)
        elif mode == "regular":
            grad = np.gradient(conc,time)
            return(grad)

        
#Uncleaned version - GRAD1
l_grad = calc_slope(final_time,final_conc,mode='log')
fig,ax2 = plt.subplots()

ax2.plot(final_time,l_grad,linestyle='-',marker='.',alpha=0.6,color='crimson')
ax2.set_xscale("log")

**Isolate the inflection points**

I do this by visualizing and handpicking the ranges that cover the two peaks. (I tried automating this but sometimes the numerical instabilities does not give the correct ratios)



In [ ]:
actual_l_grad = l_grad
actual_time = final_time

#Finding time points by just visual picking
first_peak_mask = actual_time<1
first_peak_indx = np.argmax(actual_l_grad[first_peak_mask])
first_peak = actual_time[first_peak_mask][first_peak_indx]

second_regime_mask = (actual_time>1)
second_peak_indx = np.argmax(actual_l_grad[second_regime_mask])
second_peak = actual_time[second_regime_mask][second_peak_indx]


valley_mask = (actual_time>first_peak) & (actual_time<second_peak)
min_grad = np.argmin(actual_l_grad[valley_mask])
# third_regime_mask = (actual_time>1e6)
# third_peak_indx = np.argmax(actual_l_grad[third_regime_mask])
# third_peak = actual_time[third_regime_mask][third_peak_indx]

# time_bounds = [first_peak,second_peak,third_peak]
time_bounds = [first_peak,second_peak]


print('t1 = %.3f s | t2 = %.3f s' %(time_bounds[0],time_bounds[1]))
lag_time = np.log(time_bounds[1]/time_bounds[0])

print("Trapping Factor: ",lag_time)


For higher order N-mers, follow the same routine. In case of multiple peaks, which can happend due to multiple steady-state trapped regimes, the trapping factor is calculate as the ratio between the time_point of the lastpeak to the first peak. 